In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm_laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's pybind11 module for Laplace kernel"

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
max_level = 4
fmm.configure(p, ncrit, max_level)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
nodes = fmm.build_list()

#### check colleagues of a node (after creating the interaction list)

In [12]:
print(len(nodes))
print(type(nodes[500].colleagues))   # list of nodes

37449
<class 'list'>


In [13]:
print(nodes[500].x)

for i, colleague in enumerate(nodes[500].colleagues):
    print(i, colleague.x, colleague.level, colleague.key)

[0.046871, 0.171870, 0.203121]
0 [0.015621, 0.140619, 0.171871] 5 4874
1 [0.046871, 0.140619, 0.171871] 5 4878
2 [0.078121, 0.140619, 0.171871] 5 4906
3 [0.015621, 0.171870, 0.171871] 5 4876
4 [0.046871, 0.171870, 0.171871] 5 4880
5 [0.078121, 0.171870, 0.171871] 5 4908
6 [0.015621, 0.203120, 0.171871] 5 4890
7 [0.046871, 0.203120, 0.171871] 5 4894
8 [0.078121, 0.203120, 0.171871] 5 4922
9 [0.015621, 0.140619, 0.203121] 5 4881
10 [0.046871, 0.140619, 0.203121] 5 4885
11 [0.078121, 0.140619, 0.203121] 5 4913
12 [0.015621, 0.171870, 0.203121] 5 4883
13 [0.046871, 0.171870, 0.203121] 5 4887
14 [0.078121, 0.171870, 0.203121] 5 4915
15 [0.015621, 0.203120, 0.203121] 5 4897
16 [0.046871, 0.203120, 0.203121] 5 4901
17 [0.078121, 0.203120, 0.203121] 5 4929
18 [0.015621, 0.140619, 0.234371] 5 4882
19 [0.046871, 0.140619, 0.234371] 5 4886
20 [0.078121, 0.140619, 0.234371] 5 4914
21 [0.015621, 0.171870, 0.234371] 5 4884
22 [0.046871, 0.171870, 0.234371] 5 4888
23 [0.078121, 0.171870, 0.234371] 5 

Some nodes may not have all 27 colleagues. In that case, `len(colleagues)` is still 27, but the missing colleagues are filled with `None`. For example, the level 1 node `nodes[2]` only have 8 colleagues, other entries are `None`:

In [14]:
print(nodes[2].colleagues)

[None, None, None, None, None, None, None, None, None, None, None, None, <exafmm_laplace.Node object at 0x7fe3f16df538>, <exafmm_laplace.Node object at 0x7fe3f16df570>, None, <exafmm_laplace.Node object at 0x7fe3f16df6f8>, <exafmm_laplace.Node object at 0x7fe3f16df848>, None, None, None, None, <exafmm_laplace.Node object at 0x7fe3f16df1f0>, <exafmm_laplace.Node object at 0x7fe3f16df7a0>, None, <exafmm_laplace.Node object at 0x7fe3f16dfdf8>, <exafmm_laplace.Node object at 0x7fe3f16dfdc0>, None]


#### precompute

In [15]:
fmm.precompute()

#### evaluate potentials

In [16]:
potentials = fmm.evaluate()

P2M                  : 8.7409600e-01
M2M                  : 6.8178900e-01
P2L                  : 2.3700000e-03
M2P                  : 1.9300000e-04
P2P                  : 2.6337000e-02
M2L                  : 6.0502100e+00
L2L                  : 6.4793500e-01
L2P                  : 8.8624900e-01


In [17]:
potentials[::10000]

array([7341.54480379, 6263.88674676, 8901.03960242, 7451.66377499,
       6217.8490947 , 7922.35937349, 7249.05769605, 8511.02993804,
       6380.38664327, 6926.16142091])

#### check accuracy

In [18]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 3.2261323e-10
Gradient Error       : 1.2922895e-07


#### update charges and run FMM iteratively

In [ ]:
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy